# 130. Transformer model for language understanding
#### Google Tutorial의 새로운 구현 예제가 더 깔끔하기는 하지만 예전의 Transformer 구현에서는 Multi-head attention을 직접 구현하여 Query, Key, Value의 실제 구현을 이해하기 좋은 장점이 있습니다.

<img src=https://www.tensorflow.org/images/tutorials/transformer/transformer.png width=500>

# Transformer model 작성 from scratch

- Portugal - English translation model 작성

- text genertion 및 attention 이 선수 지식임  

- Transformer model 의 핵심 idea 는 self-attention  

- Transformer 는 self-attention layer 를 쌓아서 구성 (RNN, CNN 사용 않음)    
    - Scaled dot product attention
    <img src="https://www.tutorialexample.com/wp-content/uploads/2020/10/Scaled-Dot-Product-Attention.png" width="400" />
   
    - Multi-head attention
<img src="https://wikidocs.net/images/page/159310/mha_img_original.png" width="400" />
    

- stacked attention layer 의 장점  
    - data 간의 시간적, 공간적 관계를 배제하여 multi-processing 가능  
    - RNN, CNN layer 를 거치지 않아도, 떨어져 있는 item 들이 서로의 output 에 영향을 끼칠 수 있다.
    - 긴 문장내의 상호 관계를 학습할 수 있다. (모든 sequential task 의 도전 과제)  
    
- 단점  
    - 시계열 자료의 경우 RNN 처럼 특정 time-step 의 output 계산을 위해서는 input 과 현재의 hidden state 만으로는 안되고 전체 history 가 있어야 함
    - 단어의 위치 정보를 추가해 주어야 한다. (positional encoding)

이 노트에서 모델을 학습 한 후 포르투갈어 문장을 입력하고 영어 번역을 반환 할 수 있음.

<img src="https://www.tensorflow.org/images/tutorials/transformer/attention_map_portuguese.png" width="1000" alt="Attention heatmap">

## Setup input pipeline

[TFDS](https://www.tensorflow.org/datasets) 를 이용하여 [TED Talks Open Translation Project](https://www.ted.com/participate/translate) 로 부터 [Portugese-English translation dataset](https://github.com/neulab/word-embeddings-for-nmt) 을 load.

dataset 은 약 50000 개의 training example 과 1100 개의 validation examples, 2000 개의 test example 로 구성


- 이 데이터세트는 포르투갈어/영어 문장 쌍을 생성합니다

## Text tokenization & detokenization

텍스트를 embedding에 대한 인덱스인 토큰 ID 시퀀스로 변환합니다.

이 데이터 세트에 최적화된 subword tokenizer(text.BertTokenizer)를 빌드하고 저장된 모델로 내보냅니다.

Saved_model을 다운로드하고 압축을 풀고 가져옵니다. 

tokenizer model 작성 과정은 [사용자 정의 tokenizer](https://www.tensorflow.org/text/guide/subwords_tokenizer) 참조

In [ ]:
# 사전 훈련된 Subword Tokenizer download

tf.saved_model에는 두 개의 텍스트 토크나이저가 포함되어 있습니다. 하나는 영어용이고 다른 하나는 포르투갈어용입니다. 둘 다 동일한 방법을 사용합니다.

tokenizer에 포함된 method들은 다음과 같습니다.

tokenize 메서드는 문자열 배치를 토큰 ID의 padded batch로 변환합니다. 이 방법은 구두점, 소문자를 분할하고 토큰화하기 전에 입력을 유니코드로 정규화합니다. 

detokenize 메서드는 tokenized ID를 사람이 읽을 수 있는 텍스트로 다시 변환 합니다.

lookup method는 tokenized ID를 tokenized text로 변환합니다.

여기에서 토크나이저의 "subwords"를볼 수 있습니다. 예를 들어, "searchability"라는 단어는 "search ##ability"로, "serendipity"라는 단어는 "s ##ere ##nd ##ip ##ity"로 분해됩니다. 

"Transformer is awesome."을 tokenize 하면 다음과 같습니다.

## 입력 파이프라인 설정 - (1)
훈련에 적합한 입력 파이프라인을 구축하기 위해 데이터 세트에 몇 가지 변환을 적용합니다.

tensorflow model에 입력으로 사용하기 위해 원시 text의 비정형텐서(ragged tensor)를 tensor로 변환합니다.


In [ ]:
def tokenize_pairs(pt, en):
    # Convert from ragged to dense, padding with zeros.
    # Convert from ragged to dense, padding with zeros.

다음은 데이터를 처리, 셔플 및 일괄 처리하는 간단한 입력 파이프라인입니다.

- train, validation batch pipeline 생성  
- tf.data.AUTOTUNE 은 작동하는 Network가 스스로 설정하고 Dataset을 잘 불러올 수 있게 결정하라는 것

In [ ]:
def make_batches(ds):

validation set  에서 batch 를 하나 가져와 dataset pipeline 이 잘 구성 되었는지 조회 합니다.

## Positional encoding - (2)

 - 위치 인코딩이 추가되어 문장에서 단어의 상대 위치에 대한 정보를 모델에 제공

- positional encoding 벡터를 임베딩 벡터에 추가하면 d 차원 공간에서 단어의 의미와 문장에서의 위치의 유사성에 따라 단어가 서로 더 가깝게 된다.

- [positional encoding](https://github.com/tensorflow/examples/blob/master/community/en/position_encoding.ipynb) 에서 자세한 내용 참조. 
- 위치 인코딩을 계산하는 공식은 다음과 같다.

$${PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$${PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$
<div align="center">
<li>d_model : embedding vector 의 차원 (512)</li>
<li>pos : 입력문장의 단어 순서 위치 </li>
<li>i : embedding vector 내의 차원의 index</li>
</div>

In [ ]:
def get_angles(pos, i, d_model):

In [ ]:
def positional_encoding(position, d_model):
  # 짝수 위치 (2i) 에는 sin 적용
  # 홀수 위치 (2i) 에는 cos 적용

In [ ]:
# Juggle the dimensions for the plot

## Masking  - (3)

- 두가지 mask 생성 
    - PAD mask : encoder, decoder 의 sequence batch 내의 `<PAD>` token 위치 표시
    - look ahead mask : decoder 에서 뒤쪽 단어를 보지 않기 위해 mask

#### PAD Mask
- sequence batch  내의 모든 패드 토큰 (0) 을 마스킹. 
- 이렇게 하면 모델이 패딩을 입력으로 처리하지 않는다. 
- 마스크는 패드 값 '0'이있는 위치를 나타낸다.
- 해당 위치에서 '1'을 출력하고 그렇지 않으면 '0'을 출력한다.

In [ ]:
def create_padding_mask(seq):
    # attention logit 에 padding 을 더하기 위해 extra dimension 을  추가

상수 tensor 를 생성하여 위 함수를 Test  
- shape 이 (batch size, 1, 1, seq_len) 이고, 
- 0 padding 위치가 1 로 masking 되었는지 확인

#### look-ahead 마스크
- sequence 의 뒤쪽(오른쪽) 토큰을 마스킹하는 데 사용. 
- 즉, 마스크는 예측에 사용하지 말아야 할 단어를 표시
- 예를 들어 세 번째 단어를 예측하기 위해 첫 번째와 두 번째 단어 만 사용됨을 의미
- 네 번째 단어를 예측하려면 마찬가지로 첫 번째, 두 번째 및 세 번째 단어 만 사용

In [ ]:
def create_look_ahead_mask(size):

## Scaled dot product attention - (4)

<img src="https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png" width="400" alt="scaled_dot_product_attention">

- 트랜스포머에서 사용되는 attention 함수는 Q (query), K (key), V (value)의 세 가지 벡터를 입력으로 받는다. 
- attention weight 를 계산하는 데 사용되는 공식은 다음과 같다.

$${Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

- dot-product attention 은 depth(embedding dimension) 의 제곱근에 의해 scaling 된다. scaling하는 이유는 큰 depth 값은 dot product 값을 크게 만들고 이 것을 소프트맥스 함수에 집어넣으면 기울기를 얻기 힘들게 되기 때문이다.
- 예를 들어, Q 와 K 가 표준 정규분포일 경우, matrix multiplication 은 평균 0, variance dk 가 되게 하므로 dk 의 제곱근으로 나누어주면 Q 와 K 의 matmul 은 평균 0, variance 1 이 되어 기울기를 구하기 쉽게 된다.

- mask 에 -1e9(음의 무한대에 가까움) 을 곱하여 Q와 K의 스케일 된 행렬 곱셈 (scaled-attention-logit) 에 더한다.  이렇게 하면 mask 가 1 인 경우는 softmax 에 큰 음의 입력값을 주게 되고 출력이 0 이 되도록 한다.
즉, mask 가 1 인 경우는 예측을 하면 안되기 때문이다.

- softmax를 거친 값을 value에 곱해준다면, query와 유사한 value일수록, 즉 중요한 value일수록 더 높은 값을 가지게 된다. 중요한 정보에 더 관심을 둔다는 attention의 원리에 알맞은 것임.

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  # scale matmul_qk
  # scaled_attention_logit 에 (mask * -1e9) 를 더하여 padding 위치 (masked 1) 의 값이 0 이 되도록 함
  # last axis (seq_len_k) 를 기준으로 softmax normalize 되어 scores 의 합이 1 이 되도록 함.

- K 에 대하여  softmax normalization 을 하였으므로 K 의 value (V) 는 Q 에 대해 얼마나 K 가 중요한지를 결정하게 된다. 
- 위 함수에서 return 된 output 은 attention weight 와 V (value) vector 의 matmul 인데, 이 것은 주목할 단어를 그대로 보존하고, 무관한 단어를 지워버리는 역할을 한다.

In [ ]:
def print_out(q, k, v):

- scaled dot attention test

In [ ]:
# key
# value
# query
# 두번째 key([0, 10, 0])를 query 하면 
# 두번째 value([10, 0]) 가 attention weight 1 로 반환됨

In [ ]:
# 세번째 key 를 query 하면 세번째 value가 attentin weight 1 로 반환되고 
# 같은 값을 갖는 두 key값의 평균을 반환([100, 5] 와 [1000, 6] 의 평균인 [550, 5.5])

In [ ]:
# 이 query 는 첫번째와 두번째 key를 동등하게 query 하므로, 평균값 반환
# [1, 0] 와 [10, 0] 의 평균값인 [5.5, 0]

위의 세가지 쿼리를 한번에 행렬연산. 동일한 값이 출력됨을 확인.

## Multi-head attention - (5)

<img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width="500" alt="multi-head attention">


멀티 헤드 attention 은 다음 네 부분으로 구성된다.
* linear layer 와 헤드 분할.
* scaled dot-product attention.
* multi-head 의 concatenate.
* 최종 linear layer.

각 멀티 헤드 어텐션 블록에는 Q (쿼리), K (키), V (값) 의 3 개의 입력이 있다. 이들은 선형 (Dense) 레이어를 통해 여러 헤드로 분할된다.

위에서 정의한`scaled_dot_product_attention` 을 각 헤드에 적용.(효율성을 위해 broadcast 됨). attention 단계에서 적절한 마스크를 사용해야 한다. 그런 다음 각 헤드에 대한 attention output 이 연결되고 (`tf.transpose` 및 `tf.reshape` 사용) 최종 `Dense` 레이어를 통과.

하나의 단일 attention head 를 사용 않고, Q, K 및 V는 여러 헤드로 분할되어 모델이 서로 다른 표현 공간의 다른 위치에있는 정보에 공동으로 attend 할 수 있게한다. 분할 후 각 헤드의 dimension 이 감소하므로 전체 계산 비용은 전체 dimension 으로 단일 head attention 할 때와 같다.

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    # Linear Layer
  def split_heads(self, x, batch_size):
  def call(self, v, k, q, mask):
    # scaled_attention.shape  == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)

`x`  시퀀스의 각 위치에서 `MultiHeadAttention` 은 시퀀스의 다른 모든 위치에 8 개의 attention head 를 모두 실행하여 해당 위치에서 동일한 길이의 새 output 벡터를 반환.

## Point wise feed forward network - (6)

Pointwise feed forward network 은 ReLU 활성화를 통해 두 개의 완전히 연결층으로 구성된다.

In [ ]:
def point_wise_feed_forward_network(d_model, dff):

## Encoder and decoder
<img src="https://www.tensorflow.org/images/tutorials/transformer/transformer.png" width="600" alt="transformer">

transformer model 은  표준 `sequence to sequence with attention model` 과 동일한 일반적 pattern 을 따른다.

* 입력 문장은 시퀀스의 각 단어 / 토큰에 대한 출력을 생성하는 N 개의 인코더 레이어를 통과한다.
* 디코더는 다음 단어 예측을 위해 인코더의 출력과 자체의 입력 (self-attention)에 attend 한다.

### Encoder layer - (7)

각 encoder layer 는 sublayer 들로 구성:

1.   Multi-head attention (with padding mask) 
2.   Point wise feed forward networks. 

이들 서브 계층들 각각은 그 주변을 돌아가는 residual connection(잔차 연결)을 갖고 있어서 layer normalization 을 한다. 잔차 연결은 딥 네트워크에서의 vanishing gradient 문제를 방지하는 데 도움이 된다.

각 sublayer 의 output 은 `LayerNorm(x + Sublayer(x))` 이다. normalization 은 `d_model` (last) axis 에 대해 이루어 진다. transformer 는 N encoder layer 로 구성된다.

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
  def call(self, x, training, mask):

### Decoder layer - (8)

각 decoder layer 는 다음의 sublayer 들로 구성:

1.   Masked multi-head attention (look ahead mask 와 padding mask 로 구성)
2.   Multi-head attention (with padding mask). 
    - encoder 로부터 V (value) and K (key) 의 *encoder output* 을 inputs 으로 받는다. 
    - Q (query) 는  *masked multi-head attention sublayer 의 output* 을 input 으로 받는다.
3.   Point wise feed forward networks

이들 서브 계층들 각각은 그 주변에 residual connection(잔차 연결) 을 갖고 이어서 layer normalization dmf 갖는다. 각 sublayer 의 output 은 `LayerNorm(x + Sublayer(x))` 이다. normalization 은 d_model (last) axis 에 대해 이루어 진다.

N decoder layer 로 구성된다.

Q는 디코더의 첫 번째 attention 블록에서 출력을 수신하고 K가 인코더 출력을 수신함에 따라 attention weight 는 인코더의 출력을 기반으로 디코더의 입력에 주어진 중요성을 나타낸다. 다시 말해, 디코더는 엔코더의 출력을보고 자체 출력에 self attention 하여 다음 단어를 예측한다. 


In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

### Encoder 전체 - (9)

전체 `Encoder` 는 다음으로 구성:
1.   Input Embedding
2.   Positional Encoding
3.   N encoder layers

입력은 positional 인코딩과 합쳐진 임베딩을 통해 이루어진다. 이 합산의 출력은 인코더 계층에 대한 입력이다. 인코더의 출력은 디코더에 대한 입력이다.

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
  def call(self, x, training, mask):
    # adding embedding and position encoding.

### Decoder 전체 - (10)

전체 `Decoder` 는 다음으로 구성:
1.   Output Embedding
2.   Positional Encoding
3.   N decoder layers

타겟은 positional 인코딩과 합산 된 임베딩을 통해 이루어진다. 이 합산의 출력은 디코더 층에 대한 입력이다. 디코더의 출력은 최종 linear layer 에 대한 입력이다.

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
  def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
    # x.shape == (batch_size, target_seq_len, d_model)

## Transformer 생성 - (11)

Transformer 는 인코더, 디코더 및 최종 linear layer 로 구성. 디코더의 출력은 linear layer 에 대한 입력이며 출력이 반환된다.

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
  def create_masks(self, inp, tar):
    # Encoder padding mask
    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.

## Set hyperparameters

이 예제를 작고 비교적 빠르게 유지하기 위해 *num_layers, d_model 및 dff* 의 값을 줄임. 

transformer  기본 모델에 사용 된 값은; *num_layers=6*, *d_model = 512*, *dff = 2048*. 

참고 : 아래 값을 변경하면 많은 작업에서 최첨단 모델을 얻을 수 있다.

## Optimizer

[paper](https://arxiv.org/abs/1706.03762) 의 공식에 따라 custom learning rate scheduler 와 함께 Adam optimizer 를 사용한다.



$${lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
  def __call__(self, step):

## Loss and metrics

target sequences 가 padding 되어 있으므로, loss 계산 시 `padding mask` 를 적용하는 것이 중요.

`tf.keras.losses.SparseCategoricalCrossentropy` 의 reduction=NONE 으로 지정할 경우는 shape 이 `[batch_size, d0, .. dN-1]` 이고, 그렇지 않은 경우는 scalar 이다.

In [ ]:
def loss_function(real, pred):
def accuracy_function(real, pred):

# Training and checkpointing

## Transformer model 생성

In [ ]:
# if a checkpoint exists, restore the latest checkpoint.

- target 은 tar_inp와 tar_real로 나뉜다. 
- tar_inp 은 디코더에 입력으로 전달된다. tar_real은 동일한 입력이 1 step 만큼 이동 된 것이다. tar_input 의 각 위치에서 tar_real은 예측해야 할 다음 토큰을 포함한다.
```
    sentence = "SOS A lion in the jungle is sleeping EOS"
    tar_inp =  "SOS A lion in the jungle is sleeping"
    tar_real = "A lion in the jungle is sleeping EOS"
```
- training은 teacher-forcing 기법 사용 (teacher-forcing 은 현재 time step 에서 모델이 예측하는 것과 상관없이 실제 출력을 다음 time step 으로 전달)

- Inference 단계에서 Transformer 는 자동 회귀 모델이다. 한 번에 한 부분 씩 예측하고 그 결과를 사용하여 다음 token을 예측. Transformer 가 각 단어를 예측할 때 *self-attention* 을 사용하면 입력 시퀀스의 이전 단어를보고 다음 단어를 더 잘 예측할 수 있다.

- model 이 expected output 을 미리 엿보는 것을 막기 위해 look-ahead mask 사용.

In [ ]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.
def train_step(inp, tar):

### Portugal 어가 input language 이고 English 가 target language 로 사용됨.

- Colab GPU 약 20 분 이상 소요

In [ ]:
  # inp -> portuguese, tar -> english

## 추론 실행
추론에는 다음 단계가 사용됩니다.

- 포르투갈어 토크나이저(tokenizers.pt)를 사용하여 입력 문장을 인코딩합니다. 인코더 입력입니다.

- 디코더 입력은 [START] 토큰으로 초기화됩니다.

- 패딩 마스크와 look ahead mask를 계산합니다.

- 그런 다음 디코더는 인코더 출력과 자체 출력(셀프 어텐션)을 확인하여 예측을 출력합니다.

- 예측된 토큰을 디코더 입력에 연결하고 디코더에 전달합니다.

- 이 접근 방식에서 디코더는 예측한 이전 토큰을 기반으로 다음 토큰을 예측합니다.

In [ ]:
class Translator(tf.Module):
  def __init__(self, tokenizers, transformer):
  def __call__(self, sentence, max_length=20):
    # 입력 문장은 포르투갈어이므로 시작 및 종료 토큰을 추가합니다.
    # 대상이 영어이므로 transformer에 대한 첫 번째 토큰은 english start 토큰이어야 합니다.
    # 동적 루프가 `tf.function`에 의해 추적될 수 있도록 `tf.TensorArray`가 (파이썬 list 대신) 필요합니다.
      # seq_len dimension에서 last token을 선택합니다.
      # 디코더에 input으로 제공되는 output에 predicted_id를 concatenate합니다.
    # output.shape (1, tokens)
    # `tf.function`은 루프의 마지막 iteration에서 계산된 Attention_weights를 사용하지 못하게 합니다.
    # 따라서 loop 외부에서 다시 계산합니다.

이 Translator 클래스의 인스턴스를 만들고 몇 번 시도해 봅니다.

In [ ]:
def print_translation(sentence, tokens, ground_truth):

## Attention plots

Translator 클래스는 모델의 내부 작업을 시각화하는 데 사용할 수 있는 어텐션 맵의 dictionary를 반환합니다.

In [ ]:
def plot_attention_head(in_tokens, translated_tokens, attention):
  # 토큰이 생성되었을 때의 attention을 plot 합니다. 
  # 모델이 출력에서 `<START>`를 생성하지 않았습니다. skip 합니다.

In [ ]:
# shape: (batch=1, num_heads, seq_len_q, seq_len_k)

In [ ]:
def plot_attention_weights(sentence, translated_tokens, attention_heads):

모델은 익숙하지 않은 단어에 대해서도 괜찮아 보입니다. "triceratops" 또는 "encyclopedia"는 입력 데이터 세트에 없으며 모델은 공유 어휘 없이도 거의 음역(소리나는대로 번역)하는 방법을 학습합니다.

## Export

해당 추론 모델이 작동하고 있으므로 다음에는 이를 tf.saved_model로 내보냅니다.

그렇게 하려면 이번에는 __call__ 메서드에 tf.function을 사용하여 또 다른 tf.Module 하위 클래스로 래핑합니다.

In [ ]:
class ExportTranslator(tf.Module):
  def __init__(self, translator):
  def __call__(self, sentence):

위의 tf.function에서는 출력 문장만 반환됩니다. tf.function의 엄격하지 않은 실행 덕분에 불필요한 값은 절대 계산되지 않습니다.

모델이 tf.argmax를 사용하여 예측을 디코딩하기 때문에 예측은 결정적(deterministic)입니다. 원래 모델과 저장된 모델에서 다시 로드된 모델은 동일한 예측을 제공해야 합니다.

## 요약
지금까지 positional encoding, multi-head attention, 마스킹의 중요성 및 transformer를 만드는 방법에 대해 배웠습니다.

다른 데이터 세트를 사용하여 transformer를 훈련시키십시오. 위의 하이퍼파라미터를 변경하여 the base transformer 또는 transformer XL을 생성할 수도 있습니다. 여기에 정의된 레이어를 사용하여 BERT를 생성하고 최첨단 모델을 훈련할 수도 있습니다. 또한 더 나은 예측을 위해 beam search를 구현할 수 있습니다.